# Self-Improving Agent Tutorial

## Overview
This tutorial demonstrates the implementation of a Self-Improving Agent with pure Python. The agent is designed to engage in conversations, learn from its interactions, and continuously improve its performance over time.

## Motivation
As AI systems become more integrated into our daily lives, there's a growing need for agents that can adapt and improve based on their interactions. This self-improving agent serves as a practical example of how we can create AI systems that don't just rely on their initial training, but continue to evolve and enhance their capabilities through ongoing interactions.

## Key Components

1. **Language Model**: The core of the agent, responsible for generating responses and processing information.
2. **Chat History Management**: Keeps track of conversations for context and learning.
3. **Response Generation**: Produces relevant replies to user inputs.
4. **Reflection Mechanism**: Analyzes past interactions to identify areas for improvement.
5. **Learning System**: Incorporates insights from reflection to enhance future performance.

## Method Details

### Initialization
The agent is initialized with a language model, a conversation store, and a system for managing prompts and chains. This setup allows the agent to maintain context across multiple interactions and sessions.

### Response Generation
When the agent receives input, it considers the current conversation history and any recent insights gained from learning. This context-aware approach allows for more coherent and improving responses over time.

### Reflection Process
After a series of interactions, the agent reflects on its performance. It analyzes the conversation history to identify patterns, potential improvements, and areas where it could have provided better responses.

### Learning Mechanism
Based on the reflections, the agent generates learning points. These are concise summaries of how it can improve, which are then incorporated into its knowledge base and decision-making process for future interactions.

### Continuous Improvement Loop
The cycle of interaction, reflection, and learning creates a feedback loop that allows the agent to continuously refine its responses and adapt to different conversation styles and topics.

## Conclusion
This Self-Improving Agent demonstrates a practical implementation of an AI system that can learn and adapt from its interactions. By combining the power of large language models with mechanisms for reflection and learning, we create an agent that not only provides responses but also improves its capabilities over time.

This approach opens up exciting possibilities for creating more dynamic and adaptable AI assistants, chatbots, and other conversational AI applications. As we continue to refine these techniques, we move closer to AI systems that can truly learn and grow from their experiences, much like humans do.

## Imports and Setup

First, we'll import the necessary libraries and load our environment variables.

In [2]:
from openai import OpenAI
import os
from dotenv import load_dotenv

# Initialize OpenAI client
client = OpenAI()

# Load environment variables
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")
openai_base_url = os.getenv("OPENAI_BASE_URL")

## Helper Functions

We'll define helper functions for each capability of our agent

### Chat History Management

In [8]:
class ChatMessageHistory:
    def __init__(self):
        self.messages = []

    def add_message(self, role, content):
        self.messages.append({"role": role, "content": content})

    def get_history(self):
        return self.messages

### Reflection

In [9]:
def reflect(history):
    """Generate insights from the conversation history"""
    prompt = "Based on the following conversation history, provide insights on how to improve responses:\n"
    messages = [
        {
            "role": "system",
            "content": prompt,
        }
    ]
    for message in history:
        messages.append({"role": message['role'], "content": message['content']})
    
    messages.append({"role": "system", "content": "Generate insights for improvement:"})

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        temperature=0.7,
        max_tokens=100,
    )
    return response.choices[0].message.content

### Learning

In [10]:
def learn(insights):
    """Update the agent's knowledge and behavior based on insights"""
    prompt = f"Based on these insights, update the agent's knowledge and behavior:\n{insights}\nSummarize the key points to remember:"
    messages = [
        {"role": "system", "content": "You are an AI agent that learns from its interactions."},
        {"role": "user", "content": prompt}
    ]
    
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        max_tokens=100,
        temperature=0.7
    )
    return response.choices[0].message.content

## Self-Improving Agent Class

Now we'll define our `SelfImprovingAgent` class that uses these functions.

In [13]:
class SelfImprovingAgent:
    def __init__(self):
        self.llm_model = "gpt-4o-mini"
        self.store = {}
        self.insights = ""
        
    def get_chat_history(self, session_id: str):
        """Get or create a new chat history for a given session ID"""
        if session_id not in self.store:
            self.store[session_id] = ChatMessageHistory()
        return self.store[session_id]

    def respond(self, human_input: str, session_id: str):
        """Get a response from the agent, considering the chat history"""
        history = self.get_chat_history(session_id).get_history()
        
        prompt = "You are an AI assistant. Given the conversation history, generate a response to the following input:\n"
        messages = [
            {"role": "system", "content": prompt},
        ]
        for message in history:
            messages.append({"role": message["role"], "content": message["content"]})
        messages.append({"role": "user", "content": human_input})

        response = client.chat.completions.create(
            model=self.llm_model,
            messages=messages,
            max_tokens=150,
            temperature=0.7
        )
        ai_response = response.choices[0].message.content

        # Add the user input and AI response to the chat history
        self.get_chat_history(session_id).add_message("user", human_input)
        self.get_chat_history(session_id).add_message("assistant", ai_response)
        
        return ai_response

    def reflect_and_learn(self, session_id: str):
        """Reflect on the conversation history and learn from it"""
        history = self.get_chat_history(session_id).get_history()
        self.insights = reflect(history)
        
        learned_points = learn(self.insights)
        
        self.get_chat_history(session_id).add_message("system", f"[SYSTEM] Agent learned: {learned_points}")
        
        return learned_points

## Example Usage

Let's create an instance of our agent and interact with it to demonstrate its self-improving capabilities.

In [14]:
agent = SelfImprovingAgent()
session_id = "user_123"

# Interaction 1
print("AI:", agent.respond("What's the capital of France?", session_id))

# Interaction 2
print("AI:", agent.respond("Can you tell me more about its history?", session_id))

# Learn and improve
print("\nReflecting and learning...")
learned = agent.reflect_and_learn(session_id)
print("Learned:", learned)

# Interaction 3 (potentially improved based on learning)
print("\nAI:", agent.respond("What's a famous landmark in this city?", session_id))

# Interaction 4 (to demonstrate continued improvement)
print("AI:", agent.respond("What's another interesting fact about this city?", session_id))

AI: The capital of France is Paris.
AI: Certainly! Paris has a rich and complex history that dates back over 2,000 years. Originally founded by a Celtic tribe known as the Parisii around the 3rd century BC, it was later conquered by the Romans in 52 BC and became a significant city in the Roman Empire, known as Lutetia.

In the Middle Ages, Paris grew in importance as a religious and cultural center, becoming the capital of the Kingdom of France in the 10th century. The city played a crucial role during the French Revolution in the late 18th century, as it was the site of many key events, including the storming of the Bastille in 1789.

Throughout the 19th century, Paris underwent significant changes, including

Reflecting and learning...
Learned: Here are the key points to remember for updating the agent's knowledge and behavior:

1. **Depth of Information**: Enhance responses by including detailed information about specific events, cultural developments, and significant figures in Pa